# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 
## Homework 4: Predicting College Admissions

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner

<hr style="height:2.4pt">

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css"
).text
HTML(styles)

In [ ]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


<hr style="height:2pt">

### INSTRUCTIONS

- **THIS IS AN INDIVIDUAL ASSIGNMENT**. 

- To submit your assignment follow the instructions given in Canvas.

- This assignment **must be done individually**.

- Please restart the kernel and run the entire notebook again before you submit.

- Running cells out of order is a common pitfall in Jupyter Notebooks. To make sure your code works restart the kernel and run the whole notebook again before you submit. Exceptions should be made for code with a long execution time, of course.
- We have tried to include all the libraries you may need to do the assignment in the imports statement at the top of this notebook. We strongly suggest that you use those and not others as we may not be familiar with them.
- Please use .head() when viewing data. Do not submit a notebook that is **excessively long**. 
- In questions that require code to answer, such as "calculate the $R^2$", do not just output the value from a cell. Write a `print()` function that includes a reference to the calculated value, **not hardcoded**. For example: 
```
print(f'The R^2 is {R:.4f}')
```
- Your plots should include clear labels for the $x$ and $y$ axes as well as a descriptive title ("MSE plot" is not a descriptive title; "95 % confidence interval of coefficients of polynomial degree 5" is).
<hr style="height:2pt">

## Overview and Data Description

### Predicting admissions into elite universities

In this problem set we will model the chances of high school students being accepted into two different elite undergraduate colleges (one is elite at least :) ): Harvard and Yale.  The data are provided in the file `data/college_admissions.csv` and were scraped from [collegedata.com](https://www.collegedata.com/) (where applicants volunteer to share their information).  Each observation corresponds to an applicant to one of the two different colleges (note: the same applicant may show up in two rows: once for each college).  The main response is the `admitted` variable (1 = admitted, 0 = denied), and there are are several predictors to consider:

- **id**: a unique identifier for the applicant 
- **test**: a standardized measurement of the applicants highest ACT or SAT combined score (2400 is the maximum). 
- **ap**: the number of AP tests taken
- **avg_ap**: the average score on the AP tests taken (0 if no tests were taken)
- **sat_subjects**: the number of SAT subject tests taken
- **gpa**: the unweighted GPA of applicant (max of 4.0)
- **female**:  a binary indicator for gender: 1 = female, 0 = otherwise 
- **minority**: a binary indicator for under-represented minority: 1 = minority, 0 = otherwise 
- **international**: a binary indicator for international status: 1 = international, 0 = US
- **sports**: a binary indicator for HS All-American: 1 = all-American athlete, 0 = otherwise
- **school**: a categorical variable for school applied to: "Harvard" or "Yale"
- **early_app**: a binary indicator for application type: 1 = early action, 0 = regular
- **alumni**:  a binary indicator for parents' alumni status of school: 1 = a parent is an alumnus, 0 = otherwise
- **program**: the program applied to by the student with many choices (we will not use this as a predictor)
- **add_info**: additional (optional) info provided by applicant (we will not use this as a predictor)

The main set of 12 predictors is (note: you may need to modify this list when fitting different models, and your will be replacing the `school` variable with a binary `harvard` variable early in the questions below):

```python
[
    'test','ap','avg_ap','sat_subjects','gpa','female',
    'minority','international','sports','school','early_app','alumni'
]
```

Please use this dataset to answer the following questions below.

**Important notes:**

- **Unless stated otherwise, all logistic regression models should be unregularized (use `penalty="none"`) and include the intercept (which is the default in `sklearn`).**


- **When printing your output (e.g. coefficients, accuracy scores, etc.), DO NOT just print numbers without context. Please be certain provide clarifying labels for all printed numbers and limit the number of digits showing after decimals to a reasonable length (e.g. 4 decimal points for coefficients and accuracy scores).**


- **Also be sure to practice good data science principles: always use train to do analysis and never touch the test set until the very end.**

---

<div class='exercise'><b> Question 1 [16 pts]: Data Exploration using train and basic models </b></div>

The first step is to split the observations into an approximate 80-20 train-test split.  Below is some code to do this for you (we want to make sure everyone has the same splits). It also prints the dataset's shape before splitting and after splitting. 

**IMPORTANT:** While a valid argument could be made to scale our predictors here, please **DO NOT** do so **UNTIL** it is requested of you in **question 4.1**.

**1.1** What proportion of observations were admitted overall?  What would be the classification accuracy for a baseline "naive" model where we classified ALL applicants as either admitted or not admitted using just this overall proportion to make our decision (i.e. we apply the same outcome to all applicants based on this proportion)?


**1.2** Create a binary ('dummy') variable named `harvard` that takes on the value 1 if `school == "Harvard"` and 0 otherwise. Now, explore the marginal association of each of our 12 predictors with whether or not an applicant is admitted into the college they applied (`admitted`).  Create a separate **visual** for each predictor to investigage their relationship with college admissions.  **Suggestion:** place these 12 visuals in a *matrix* of subplots with 3 columns and 4 rows.

**Note:** We will be using our dummified `harvard` predictor instead of `school` throughout the remainder of this problem set.

**1.3** Based on the visuals above, which predictor seems to have the most potential for predicting `admitted`? Why do you think this it the best potential single predictor?


**1.4** Fit a logistic regression to predict `admitted` from `harvard` (call it `logit1_4`).  Interpret the coefficient estimates: which college is estimated to be easier to get into?  What are the estimated chances of getting into each school?


**1.5** Create a contingency table between `admitted` and `harvard`.  Use this table to calculate and confirm the coefficient estimates in the `logit1_4` model (both the intercept and slope).


**1.6** Compare the estimated probabilities of being admitted into the schools to the overall acceptance rate (as seen [here](https://www.ivycoach.com/2023-ivy-league-admissions-statistics/)).  Why may what you've observed in this comparison be the case?


In [ ]:
#############################
## DO NOT MODIFY THIS CODE ##
#############################

college = pd.read_csv('data/college_admissions.csv')
np.random.seed(121)

college_train, college_test, = train_test_split(
    college, test_size=0.2,  random_state = 121, shuffle=True, stratify = college['school']
)

print(college.shape)
print(college_train.shape, college_test.shape)

<div class='exercise-r'>  
 
**1.1** What proportion of observations were admitted overall?  What would be the classification accuracy for a baseline "naive" model where we classified ALL applicants as either admitted or not admitted using just this overall proportion to make our decision (i.e. we apply the same outcome to all applicants based on this proportion)?
 
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**1.2** Create a binary ('dummy') variable named `harvard` that takes on the value 1 if `school == "Harvard"` and 0 otherwise. Now, explore the marginal association of each of our 12 predictors with whether or not an applicant is admitted into the college they applied (`admitted`).  Create a separate **visual** for each predictor to investigage their relationship with college admissions.  **Suggestion:** place these 12 visuals in a *matrix* of subplots with 3 columns and 4 rows.
 
 **Note:** We will be using our dummified `harvard` predictor instead of `school` throughout the remainder of this problem set.
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**1.3** Based on the visuals above, which predictor seems to have the most potential for predicting `admitted`? Why do you think this it the best potential single predictor?
 
 
 </div>

**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
 
**1.4** Fit a logistic regression to predict `admitted` from `harvard` (call it `logit1_4`).  Interpret the coefficient estimates: which college is estimated to be easier to get into?  What are the estimated chances of getting into each school?
 
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
 
**1.5** Create a contingency table between `admitted` and `harvard`.  Use this table to calculate and confirm the coefficient estimates in the `logit1_4` model (both the intercept and slope).
 
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
    
**1.6** Compare the estimated probabilities of being admitted into the schools to the overall acceptance rate (as seen [here](https://www.ivycoach.com/2023-ivy-league-admissions-statistics/)).  Why may what you've observed in this comparison be the case?
</div>

**INTERPRETATION:**

*Your answer here*

---

<div class='exercise'><b> Question 2 [18 pts]: Interpretable Modeling </b></div>

**2.1** Fit a logistic regression model to predict `admitted` from `test` alone (call it `logit2_1`).  Print out the coefficient estimates (both intercept and slope) along with the classification accuracies for this model (on both train and test data). 

**2.2** What are the estimated chances of an applicant being admitted with an *average* `test` score of 2200?  What about if they had a perfect test score of 2400?  What test score would be needed to have a 50-50 chance of being admitted?

**2.3**  Fit a logistic regression model to predict `admitted` from `test` and `avg_ap` (call it `logit2_3`).  Print out the coefficient estimates (both intercept and slope) along with the classification accuracies for this model (on both train and test data). 

**2.4** Interpret the coefficient estimates in `logit2_3` (not the intercept) and compare the coefficient estimate for `test` to the one from `logit2_1`.  Why has this estimate changed?

**Hint:** You may want to inspect the relationship between `test` and `avg_ap` to help get a better sense for what might be happening here.

**2.5** Interpret and compare the classification accuracies for the two models (`logit2_1` and `logit2_3`).  Explain why these accuracies are the same or different, and what about the data makes these accuracies so similar of different? 


<div class='exercise-r'>  
 
**2.1** Fit a logistic regression model to predict `admitted` from `test` alone (call it `logit2_1`).  Print out the coefficient estimates (both intercept and slope) along with the classification accuracies for this model (on both train and test data).
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**2.2** What are the estimated chances of an applicant being admitted with an *average* `test` score of 2200?  What about if they had a perfect test score of 2400?  What test score would be needed to have a 50-50 chance of being admitted?
 
 </div>

In [ ]:
# your code here


*Your answer here*

In [ ]:
# your code here


**Interpretation:**

*Your answer here*

<div class='exercise-r'>  
 
**2.3**  Fit a logistic regression model to predict `admitted` from `test` and `avg_ap` (call it `logit2_3`).  Print out the coefficient estimates (both intercept and slope) along with the classification accuracies for this model (on both train and test data).
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**2.4** Interpret the coefficient estimates in `logit2_3` (not the intercept) and compare the coefficient estimate for `test` to the one from `logit2_1`.  Why has this estimate changed?
 
 **Hint:** You may want to inspect the relationship between `test` and `avg_ap` to help get a better sense for what might be happening here.
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
    
**2.5** Interpret and compare the classification accuracies for the two models (`logit2_1` and `logit2_3`).  Explain why these accuracies are the same or different, and what about the data makes these accuracies so similar of different?
</div>

**INTERPRETATION:**

*Your answer here*

---

<div class='exercise'><b> Question 3 [30 pts]: Harvard and Yale? </b></div>


**3.1** Fit a logistic regression model (call it `logit3_1`) to predict `admitted` from 7 predictors: `['harvard', 'test', 'ap', 'avg_ap', 'gpa', 'female', 'minority']`.  Output and interpret the coefficient estimates for the binary predictors in this model.

**Hint:** If you have convergence warnings, increasing the maximum number of iterations to 500 (`max_iter=500`) will likely solve the issue.

**3.2** Fit a logistic regression model (call it `logit3_2`) to predict `admitted` from 3 predictors: `['harvard', 'test', 'ap']` along with the 2 interaction terms: `harvard` with `test` and `harvard` with `ap`. Name the columns for these interaction terms something sensible.  Print out the coefficient estimates for this model.

**3.3** Simplify and write out mathematically the above model from question 3.2 for 2 applicants: (1) someone who is applying to Harvard and for (2) someone who is applying to Yale (keep `test` and `ap` as the unknown $X$s).  The basic framework given to you below may be helpful:

$$ \ln \left( \frac{P(Y=1)}{1-P(Y=1)} \right) = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p $$

**Note:** All of your mathematical statements should be written out in your markdown cells [using LaTeX](https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd). Do not insert images of handwritten math!

**3.4** Determine two classification boundaries mathematically for the model in the previous part (using the estimated coefficients): What range of values of `test` as a function of `ap` would an applicant be predicted to more likely than not be admitted into the college they applied? If a student scored a perfect 2400 on `test`, what is the minimum number of AP tests they should take in order to have a better than 50% chance of being admitted into Harvard?

**3.5** Create two separate scatterplots (one for Harvard applicants and one for Yale applicants) with the predictor `test` on the y-axis and `ap` on the x-axis where `admitted` is color-coded and the marker denotes train vs. test data.  Then add the appropriate classification boundary from the previous part.  Compare these two plots (both the location of the boundaries and where the points lie around these boundaries).

**Note:** As always, please be certain (a) your plot is titled, (b) everything is clearly labeled, and (c) the plot itself is formatted in a manner that makes it easy to read and interpret.

**3.6** Fit a logistic regression model (call it `logit3_6`) to predict `admitted` from 4 predictors: `['harvard', 'test', 'female', 'minority']` along with 2 interaction terms: `harvard` with `female` and `harvard` with `minority`.  Print out the coefficient estimates for this model.

**3.7** Interpret the coefficients associated with `female` and `minority` (the two main effects AND the two interaction terms).

**3.8** Based on this sample, how does it appear that Harvard and Yale compare in admitting these groups?  Why would it be wrong to take this interpretation as truth?

<div class='exercise-r'>  
 
**3.1** Fit a logistic regression model (call it `logit3_1`) to predict `admitted` from 7 predictors: `['harvard', 'test', 'ap', 'avg_ap', 'gpa', 'female', 'minority']`.  Output and interpret the coefficient estimates for the binary predictors in this model.
 
 **Hint:** If you have convergence warnings, increasing the maximum number of iterations to 500 (`max_iter=500`) will likely solve the issue.
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
 
**3.2** Fit a logistic regression model (call it `logit3_2`) to predict `admitted` from 3 predictors: `['harvard', 'test', 'ap']` along with the 2 interaction terms: `harvard` with `test` and `harvard` with `ap`. Name the columns for these interaction terms something sensible.  Print out the coefficient estimates for this model.
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**3.3** Simplify and write out mathematically the above model from question 3.2 for 2 applicants: (1) someone who is applying to Harvard and for (2) someone who is applying to Yale (keep `test` and `ap` as the unknown $X$s).  The basic framework given to you below may be helpful:
 
 $$ \ln \left( \frac{P(Y=1)}{1-P(Y=1)} \right) = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p $$
 
 **Note:** All of your mathematical statements should be written out in your markdown cells [using LaTeX](https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd). Do not insert images of handwritten math!
 
 </div>

**ANSWER:**

*Note: mathematical statements should be written using LaTeX.*

*Your answer here*

<div class='exercise-r'>  
 
**3.4** Determine two classification boundaries mathematically for the model in the previous part (using the estimated coefficients): What range of values of `test` as a function of `ap` would an applicant be predicted to more likely than not be admitted into the college they applied? If a student scored a perfect 2400 on `test`, what is the minimum number of AP tests they should take in order to have a better than 50% chance of being admitted into Harvard?
 
 </div>

**ANSWER:**

*Note: mathematical statements should be written using LaTeX.*

*Your answer here*

<div class='exercise-r'>  
 
**3.5** Create two separate scatterplots (one for Harvard applicants and one for Yale applicants) with the predictor `test` on the y-axis and `ap` on the x-axis where `admitted` is color-coded and the marker denotes train vs. test data.  Then add the appropriate classification boundary from the previous part.  Compare these two plots (both the location of the boundaries and where the points lie around these boundaries).
 
 **Note:** As always, please be certain (a) your plot is titled, (b) everything is clearly labeled, and (c) the plot itself is formatted in a manner that makes it easy to read and interpret.
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
 
**3.6** Fit a logistic regression model (call it `logit3_6`) to predict `admitted` from 4 predictors: `['harvard', 'test', 'female', 'minority']` along with 2 interaction terms: `harvard` with `female` and `harvard` with `minority`.  Print out the coefficient estimates for this model.
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**3.7** Interpret the coefficients associated with `female` and `minority` (the two main effects AND the two interaction terms).
 
 </div>

**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
    
**3.8** Based on this sample, how does it appear that Harvard and Yale compare in admitting these groups?  Why would it be wrong to take this interpretation as truth?
</div>

**INTERPRETATION:**

*Your answer here*

---

<div class='exercise'><b> Question 4 [24 pts]: Building Predictive Models for admitted </b></div>

**4.1** You were instructed to NOT scale predictors in the prior sections above. The primary reason for this was to focus instead on the interpretability of our logistic regression coefficients. However, as we're sure you noticed, the numeric scale among our different predictors varies greatly (i.e. `test` values are in the 1,000's while others are much, much smaller). In practice, we might want to put our predictors all on a similar scale, particularly for regression and/or distance-based algorithms such as $k$-NN classification. (1) Please explain why scaling under these circumstances might be important. Then, (2) actually apply standardized scaling to all of the **non-binary** predictors in our original set of 12 predictors (for both the training and test sets).

**Note:** These scaled predictors should be used instead of the original unscaled versions of the predictors for the remainder of this problem set.

**4.2** Fit a well-tuned $k$-NN classification model with main effects of all 12 predictors in it (call it `knn_model`).  Use `ks = [1,3,5,9,15,21,51,71,101,151,201]` and 3-fold cross-validation. Plot, on a single set of axes, your resulting cross-validation mean training and mean validation scores at each value $k$. Then, report your chosen $k$ and the classification accuracy on train and test for your final fitted model.

**4.3** Fit the full logistic regression model with main effects of all 12 predictors in it (call it `logit_full`). Print out the coefficient estimates and report the classification accuracy on train and test for this model.

**4.4** Fit a well-tuned Lasso-like logistic regression model from all 12 predictors in it (call it `logit_lasso`). Use `Cs = [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3,1e4]` and 3-fold cross-validation.  Print out the coefficient estimates and report the classification accuracy on train and test for this model.

**4.5** Which predictors were deemed important in `logit_lasso`?  Which were deemed unimportant? 

**4.6** Fit a well-tuned Lasso-like logistic regression model with all important predictors from `logit_lasso` and all the 2-way interactions between them (call it `logit_lasso_interact`).  Again use `Cs = [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3,1e4]` and 3-fold cross-validation. Report the classification accuracy on train and test for this model.

**4.7** How many of the predictors in our `logit_lasso_interact` model were deemed important and unimportant? (Feel free to just report on the number of them found to be important and unimportant. There is no need to list them all here.)

**Hint:** If you have convergence warnings, increasing the maximum number of iterations to 500 (`max_iter=500`) will likely solve the issue.

<div class='exercise-r'>  
 
**4.1** You were instructed to NOT scale predictors in the prior sections above. The primary reason for this was to focus instead on the interpretability of our logistic regression coefficients. However, as we're sure you noticed, the numeric scale among our different predictors varies greatly (i.e. `test` values are in the 1,000's while others are much, much smaller). In practice, we might want to put our predictors all on a similar scale, particularly for regression and/or distance-based algorithms such as $k$-NN classification. (1) Please explain why scaling under these circumstances might be important. Then, (2) actually apply standardized scaling to all of the **non-binary** predictors in our original set of 12 predictors (for both the training and test sets).
 
 **Note:** These scaled predictors should be used instead of the original unscaled versions of the predictors for the remainder of this problem set.
 
 </div>

**INTERPRETATION:**

*your answer here*

In [ ]:
# your code here


<div class='exercise-r'>  
 
**4.2** Fit a well-tuned $k$-NN classification model with main effects of all 12 predictors in it (call it `knn_model`).  Use `ks = [1,3,5,9,15,21,51,71,101,151,201]` and 3-fold cross-validation. Plot, on a single set of axes, your resulting cross-validation mean training and mean validation scores at each value $k$. Then, report your chosen $k$ and the classification accuracy on train and test for your final fitted model.
 
 </div>

In [ ]:
np.random.seed(121) # Do not delete or modify this line of code

# your code here


<div class='exercise-r'>  
 
**4.3** Fit the full logistic regression model with main effects of all 12 predictors in it (call it `logit_full`). Print out the coefficient estimates and report the classification accuracy on train and test for this model.
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**4.4** Fit a well-tuned Lasso-like logistic regression model from all 12 predictors in it (call it `logit_lasso`). Use `Cs = [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3,1e4]` and 3-fold cross-validation.  Print out the coefficient estimates and report the classification accuracy on train and test for this model.
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**4.5** Which predictors were deemed important in `logit_lasso`?  Which were deemed unimportant?
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**4.6** Fit a well-tuned Lasso-like logistic regression model with all important predictors from `logit_lasso` and all the 2-way interactions between them (call it `logit_lasso_interact`).  Again use `Cs = [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3,1e4]` and 3-fold cross-validation. Report the classification accuracy on train and test for this model.
 
 </div>

In [ ]:
# your code here


<div class='exercise-r'>  
 
**4.7** How many of the predictors in our `logit_lasso_interact` model were deemed important and unimportant? (Feel free to just report on the number of them found to be important and unimportant. There is no need to list them all here.)
 
 **Hint:** If you have convergence warnings, increasing the maximum number of iterations to 500 (`max_iter=500`) will likely solve the issue.
 </div>

In [ ]:
# your code here


---

<div class='exercise'><b> Question 5 [12 pts]: Evaluating Results </b></div>


**5.1** Which of the 4 models in problem 4 perform the best based on classification accuracy?  Which performs the worst? Based on these accuracies, how do these models perform compared to your baseline "naive" model back in question 1.1?

**5.2** Draw the four ROC curves on the same set of axes using the test data.  How do these ROC curves compare?  Do the ROC curves support that the best model identified in question 5.1 is better than the worst model identified in 5.1?  How do you know?

**5.3** Calculate and report AUC for all 4 models.  Do the rankings of these 4 models based on AUC match those for classification accuracy?  Why do you think this is the case?

**5.4** If you were to use one of these 4 models to present as a prediction model for the website [collegedata.com](https://www.collegedata.com/), which would you use?  What may be the biggest issue if this was a publicly available tool for college applicants to use to determine their chances of getting into Harvard and/or Yale?


<div class='exercise-r'>  
 
**5.1** Which of the 4 models in problem 4 perform the best based on classification accuracy?  Which performs the worst? Based on these accuracies, how do these models perform compared to your baseline "naive" model back in question 1.1?
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
 
**5.2** Draw the four ROC curves on the same set of axes using the test data.  How do these ROC curves compare?  Do the ROC curves support that the best model identified in question 5.1 is better than the worst model identified in 5.1?  How do you know?
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
 
**5.3** Calculate and report AUC for all 4 models.  Do the rankings of these 4 models based on AUC match those for classification accuracy?  Why do you think this is the case?
 
 </div>

In [ ]:
# your code here


**INTERPRETATION:**

*Your answer here*

<div class='exercise-r'>  
    
**5.4** If you were to use one of these 4 models to present as a prediction model for the website [collegedata.com](https://www.collegedata.com/), which would you use?  What may be the biggest issue if this was a publicly available tool for college applicants to use to determine their chances of getting into Harvard and/or Yale?
</div>

**INTERPRETATION:**

*Your answer here*